# Frequently record weather data in your EC2 instance 

## Goals

In this case, you will learn how to set up an automation script in the cloud. You will use Amazon's Elastic Compute Cloud (```EC2```) to automatically retrieve weather data via an application programming interface (```API```).

Throughout the case study, you will configure and utilize a cloud resource meant for computational tasks. You should become familiar with the different basic options and capabilities specific to AWS while also getting acustomed to working on a remote machine through your terminal ```SSH```.

## Introduction

**Business Context.** You are an independent consultant working for multiple clients. One of your clients specializes in environmental research and they have approached you with a new requirement. They would like to keep track of the weather of New York and record how it changes over a period of time. They would like your help on fetching the weather data every hour from an API and storing that in an EC2 instance. Along with storing the weather data, they would also like you to generate certain plots and store them in the EC2 machine as well.

**Business Problem.** Your task is to write Python scripts that would fetch the weather data from API, parse the data and store it in CSV files in the same instance. You would also generate certain plots as mentioned below and store that as well in the same instance.

**Analytical Context.** The client wants you to fetch the data from [OpenWeatherMap API](https://openweathermap.org/api). You would be registering to this service, and using the API Key from the website to pull out the weather data every hour. The steps to be followed are:

- Register on the Open Weather Map website and receive the API Key
- Write a Python script that fetches the weather data and stores it in a json file
- Write another Python script that reads the stored json file, parses it and stores it as a CSV file. The following steps need to be done for parsing:
    - The datetime options are returned in timestamp format. That has to be converted to a proper human readable format.
- After the CSV file is stored, provide the following:
    - The minimum and maximum temperature in the next 48 hours
    - A line graph of how the temperature would change in the next 48 hours
    - A line graph of how the wind speed will change across 48 hours
- Once the Python scripts are done, create a bash script that calls these functions.
- Setup a cron job on the machine to execute every hour. The cron job should call the bash script, which will in turn call these 2 Python scripts.

## Launching an EC2 instance on AWS

### What is Amazon EC2?

Virtual servers offered as a web-service that are highly configurable, scalable, and accessible via a point-and-click management console on AWS. Amazon has eliminated much of the heavy lifting traditionally required to set up the hardware, software, memory, networking, permissions, storage, security, etc. needed for dedicated computation while maintaining an abundant set of configuration options allowing you to tailor your server to your specific needs. 

### When to use EC2 as a Data Scientist?

- For computationally expensive tasks that your personal machine cannot handle
- For time consuming tasks you want to run in parallel without sacrificing performance on your personal machine
- For repetitive tasks that need to run automatically -- based on time, constraints, or external signals from other web-services or systems. 

### What are the configuration options?

Too many to fully explore each in detail as part of this case but as we walk through setting up and connecting to an instance we will touch upon the major categories of options available and provide resources for further exploration.

#### OK. Let’s set one up!

1. Start by signing in to [AWS](https://signin.aws.amazon.com/signin?redirect_uri=https%3A%2F%2Fconsole.aws.amazon.com%2Fconsole%2Fhome%3Fnc2%3Dh_ct%26src%3Dheader-signin%26state%3DhashArgs%2523%26isauthcode%3Dtrue&client_id=arn%3Aaws%3Aiam%3A%3A015428540659%3Auser%2Fhomepage&forceMobileApp=0) using your root user credentials. If you have not created your own AWS account, please consult with your TA to obtain the relevant instructions or see precase_22.1 for instructions on signing up for your own AWS account.

2. Navigate to the EC2 Management Console (Services > Compute > EC2).  
![Step 1](aws_ec2_img1.png)

3. Scroll down and click the “Launch Instance” button.  
![Step 2](aws_ec2_img2.png)

4. You are now shown a wizard that will walk you through the major categories of options required to configure your instance. To begin, we must choose an Amazon Machine Image (AMI). This is essentially a template that describes the operating system, application server, and any additional software packages we want pre-installed on our virtual server. There are many AMI’s, some created and supported by Amazon, others by community users, and many that have been optimized for specific purposes so we urge you to read the descriptions of each. For the sake of this case, we will choose **Amazon Linux AMI 2018.03.0 (HVM), SSD Volume Type**
![Step 4](aws_ec2_img3.png)

5. Next, we are asked to specify the instance type. There is a wide selection of types which vary in CPU size, memory, storage, and networking capacity and are broadly organized into four families: Storage optimized, Memory optimized, Compute optimized, or General Purpose. For now we’re going to choose the ```t2.micro``` General Purpose instance which is **free tier eligible**.  
![Step 6](aws_ec2_img4.png)

6. For further configuration, you can explore steps 3-7 but for now let’s accept the default configuration and click the “Review and Launch” button followed by “Launch”. Keep in mind that launching any services or using certain configuration options not explicitly labelled as  Free tier eligible  will result in charges on your account.  
![Step 6](aws_ec2_img5.png)

7. Select “Create a new key pair”, name it, download it, and then Launch the instance.  
![Step 7](aws_ec2_img6.png)

8. While the instance is launching, we need to set the permission to the .pem file to be 400 for SSH to work. Ignore this if you are using Windows.

```bash
chmod 400 <path to .pem file>
```

9. Now lets connect to your freshly launched instance using SSH and the key pair you just downloaded by following steps outlined [here](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AccessingInstancesLinux.html). For Windows machines, you will need PuTTY installed and then will need to follow [these](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/putty.html) steps. PuTTY is a Windows tool used thats acts as an ```ssh``` terminal to give you access to remote machines.

10. After launching is completed, scroll down and click “View Instance” to view information about your instance including its public DNS. This will be used along with the key pair to connect to your instance using the command below.

```bash
ssh -i <path to .pem file> ec2-user@<public dns of your ec2 instance>
```

## Install Dependencies

Once you have connected to the machine via SSH, let's first check if we have the required Python interpreter in the machine. Check if python3 exists, by typing `python3` on the shell. If Python3 exists, you will see the python shell. If not, install Python3 with the following command:

    sudo yum update
    sudo yum install python36

Next, let's install the dependencies that we are going to use in this case. For installing python dependencies, we need pip. Let's install pip with the below command

    sudo yum install python36-pip
    

The above command will install pip3, which installs the dependencies for the Python3. Using this, we are going to install the dependent python libraries that we are going to use for today's case.

    pip-3.6 install --user requests pandas matplotlib

## Registering with the OpenWeatherMap API

In order to register with the OpenWeatherMap API, visit - https://home.openweathermap.org/users/sign_up and Sign up. Once you register on the site and verify your email address, you can login to the site and see the API Key in the `API Keys` tab, in the home page after login.

## Fetch data from OpenWeatherMap API

Let's write a Python script to fetch the weather data from Open WeatherMap API. The API documentation for Open WeatherMap can be found here - https://openweathermap.org/api/hourly-forecast. We will be using this forecast endpoint to fetch the current weather conditions of New York. When calling this endpoint, we would have to provide the latitude and longitude of the location for which we need the weather status. You will need to pass the coordinates of New York. Which are - `40.7128` and `-74.0060`.

In your home folder, let's create a file named `fetch_weather_data.py` which will fetch the weather data and store it in a json file locally. The json file should be stored in the following folder structure - `openweathermap_data/raw_data/`. The file name should be of the following format - `weather_{datetime}.json`. Where datetime is the time for which the data is fetched. It can be found inside the `currently` key in the response.

The file should also be written to `openweathermap_data/raw_data/weather.json`, for the second script to be able to easily find the latest file to read and execute.

The sample of the file can be found below.

In your ssh terminal, create and open a new file with the following command:

    nano fetch_weather_data.py

In [ ]:
import datetime
import json
import os
import requests

OPENWEATHERMAP_API_KEY = '6be5b5fb9384186d26fc8180512ba400' # replace with '<YOUR-OPENWEATHERMAP_API-KEY>'
WEATHER_LOCATION_LATITUDE = '40.7128'
WEATHER_LOCATION_LONGITUDE = '-74.0060'

# First, check if the data folder is present. If not, create it
data_folder = 'openweathermap_data/raw_data'
if not os.path.isdir(data_folder):
    os.makedirs(data_folder)
    
# Generate the URL
url = 'https://api.openweathermap.org/data/2.5/onecall?lat={0}&lon={1}&appid={2}'.format(
    WEATHER_LOCATION_LATITUDE, WEATHER_LOCATION_LONGITUDE, OPENWEATHERMAP_API_KEY)
response = requests.get(url)
weather_data = response.json()
current_datetime = datetime.datetime.fromtimestamp(weather_data['current']['dt']).strftime('%Y_%m_%d_%H_%M')
filename = os.path.join(data_folder, 'weather_{0}.json'.format(current_datetime))
with open(filename, 'w+') as f:
    f.write(json.dumps(weather_data))
    
latest_filename = os.path.join(data_folder, 'weather.json')
with open(latest_filename, 'w+') as f:
    f.write(json.dumps(weather_data))

## Parsing and visualizing the raw data

For the next section, we will be parsing the raw json data to convert it to a pandas dataframe, perform some operations on it. After that, we will be writing the csv to a local file and then perform operations on it. We will be:

- Calculating the predicted max, min and average temperature for the next 48 hours
- Calculate the predicted max, min and average wind speed for the next 48 hours
- Draw a line graph on how the temperature would vary in the next 48 hours
- Draw a line graph on how the wind speed would vary in the next 48 hours.

The output of each section should be stored to a file.

For each of the operations, we will be writing functions here and you will have to create a python file in the EC2 instance that calls all these functions.

### Parse raw data

In this section, we will create a function that will read the latest json file (with the name `weather.json`), and convert the time from timestamp into a proper datetime object.

In [ ]:
import datetime
import json
import pandas as pd

current_date_time = None

def get_latest_weather_data():
    global current_date_time
    filename = 'openweathermap_data/raw_data/weather.json'
    with open(filename) as f:
        data = json.load(f)
    df = pd.DataFrame(data['hourly'])
    
    # Convert time into a proper datetime object
    df['time'] = pd.to_datetime(df['dt'], unit='s')
    
    # Set the current date time of the data in the global variable. This will be accessed later.
    current_date_time = datetime.datetime.fromtimestamp(data['current']['dt']).strftime('%Y_%m_%d_%H_%M')
    return df

df = get_latest_weather_data()

### Write parsed dataframe to CSV file

With the data parsing done, let's write the dataframe to a csv file. Similar to how there is a folder for the raw_data, we need to create a folder for the output data. We also need to create a folder inside output data for each run, as each run creates multiple output files.

Once the folders are created, write the dataframe to a file called `output.csv`

In [ ]:
import os

def write_csv_file(dataframe):
    folder_name = 'openweathermap_data/output_data/{0}'.format(current_date_time)
    if not os.path.isdir(folder_name):
        os.makedirs(folder_name)
        
    filename = os.path.join(folder_name, 'output.csv')
    dataframe.to_csv(filename, index=False)
    
write_csv_file(df)

### Max, Min and Average data

Next, as per the requirements, is to calculate the max, min and average values of temperature and windspeed. We need to write those values to a different csv file. Let's write a function that calculates and returns the max, min and average of the temperature column. Store the returned value in a list, as we want to save the stats for both temperature and windspeed in the same file.

In [ ]:
def calculate_temperature_stats(dataframe):
    temperature_data = {'stat': 'temp'}
    temperature_data['max'] = dataframe['temp'].max()
    temperature_data['min'] = dataframe['temp'].min()
    temperature_data['average'] = dataframe['temp'].mean()
    return temperature_data

stats = []
stats.append(calculate_temperature_stats(df))

### Exercise 1:

Write a function to calculate the min, max and average stats of windspeed similar to temperature. Append the stats to the same list, and write that to a csv file named `stats.csv` in the same folder output file was written to.

**Answer.**

-------------

## Graphs

The next operation is to plot the variations of temperature across time. We'll use matplotlib to plot the graph and write it to a file in the output folder.

In [ ]:

import matplotlib.pyplot as plt


def plot_temperature(dataframe):
    folder_name = 'openweathermap_data/output_data/{0}'.format(current_date_time)
    filename = os.path.join(folder_name, 'temperature.png')
    df.plot.line(x='time', y='temp')
    plt.savefig(filename)
    
plot_temperature(df)

### Exercise 2:

Similar to how the temperature variations were plotted and save to a file, plot the windspeed variations and write the output to `windspeed.png` in the output folder.

**Answer.**

-------------

## Compile the functions into a single file

We have been performing the operations one after the other in the above steps. Create a file called `process_weather_data.py` in your EC2 instance that contains all the functions we created so far. The file should look something like below:

In [ ]:
import datetime
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

current_date_time = None


def get_latest_weather_data():
    global current_date_time
    filename = 'openweathermap_data/raw_data/weather.json'
    with open(filename) as f:
        data = json.load(f)
        
    df = pd.DataFrame(data['hourly'])
    
    # Convert time into a proper datetime object
    df['time'] = pd.to_datetime(df['dt'], unit='s')
    
    # Set the current date time of the data in the global variable. This will be accessed later.
    current_date_time = datetime.datetime.fromtimestamp(data['current']['dt']).strftime('%Y_%m_%d_%H_%M')
    return df


def write_csv_file(dataframe):
    folder_name = 'openweathermap_data/output_data/{0}'.format(current_date_time)
    if not os.path.isdir(folder_name):
        os.makedirs(folder_name)
        
    filename = os.path.join(folder_name, 'output.csv')
    dataframe.to_csv(filename, index=False)
    

def calculate_temperature_stats(dataframe):
    temperature_data = {'stat': 'temp'}
    temperature_data['max'] = dataframe['temp'].max()
    temperature_data['min'] = dataframe['temp'].min()
    temperature_data['average'] = dataframe['temp'].mean()
    return temperature_data


def calculate_windspeed_stats(dataframe):
    temperature_data = {'stat': 'wind_speed'}
    temperature_data['max'] = dataframe['wind_speed'].max()
    temperature_data['min'] = dataframe['wind_speed'].min()
    temperature_data['average'] = dataframe['wind_speed'].mean()
    return temperature_data


def plot_temperature(dataframe):
    folder_name = 'openweathermap_data/output_data/{0}'.format(current_date_time)
    filename = os.path.join(folder_name, 'temperature.png')
    df.plot.line(x='time', y='temp')
    plt.savefig(filename)
    


def plot_windspeed(dataframe):
    folder_name = 'openweathermap_data/output_data/{0}'.format(current_date_time)
    filename = os.path.join(folder_name, 'wind_speed.png')
    df.plot.line(x='time', y='wind_speed')
    plt.savefig(filename)
    
    
if True:  #__name__ == '__main__':
    df = get_latest_weather_data()
    write_csv_file(df)
    stats = []
    stats.append(calculate_temperature_stats(df))
    stats.append(calculate_windspeed_stats(df))

    # Write the stats to a csv file
    folder_name = 'openweathermap_data/output_data/{0}'.format(current_date_time)
    filename = os.path.join(folder_name, 'output.csv')
    pd.DataFrame(stats).to_csv(filename, index=False)
    plot_temperature(df)
    plot_windspeed(df)

## Create bash script and setup cronjob

Now that we have both the Python files ready, create a new file called `process_weather_data.sh` and call both the Python files one after the other. The created file should have these lines:

    cd ~
    python36 fetch_weather_data.py
    python36 process_weather_data.py
    
    
After the file is created, convert the file to an executable format:

    chmod +x process_weather_data.sh
    
Now this file is executable and can be run as a bash script. Let's test it by removing the `openweathermap_data` folder and running the file to make sure everything works fine.

    rm -rf openweathermap_data
    ./process_weather_data.sh
    
Make sure the openweathermap_data folder is created with raw_data and output_data folders inside it.

If the file works as expected, let's create a cronjob that would execute the file every hour, so that the data gets stored in the instance every hour. Some basics on cron jobs can be found here - https://www.ostechnix.com/a-beginners-guide-to-cron-jobs/. In order to edit the cron jobs file, type the following command:

    crontab -e
    
This will open a file that shows the list of cronjobs configured. Add the following line to the end of the file:

    0 * * * * /home/ubuntu/process_weather_data.sh
    
Once the line is added, you can save and exit out of the file. We have now successfully configured the system to execute both the python scripts every hour, and the timestamped file and folder names will make sure we get access to hourly data even long after they are fetched.

## Conclusions

In this case, we learned how to use requests library to fetch data from API, store it in a file, manipulate it to get some proper information out of it. We also learned about cronjobs and how they can be useful to schedule commands to be executed at specific time periods. 

We had to make sure the data fetched every time is properly timestamped, as the data should not be overwritten when the process runs next time.

## Takeaways

In this case, we created a basic automation script and hosted it in the cloud using an Amazon AWS ```EC2``` instance. We know know how to:
    
1. Configure and launch an EC2 instance
2. Connect to the instance using SSH
3. Edit files remotely
4. Schedule routine tasks using ```crontab```
    
In the future, we will learn to augment ```EC2``` with storage provided by AWS's ```RDS```. Your role as a data professional should be to familiarize yourself with the different cloud services available at the major players and know when to use a service that is best suited for your use case. In this case, as we only had a simple computational task, we opted for a small (and free) computation cloud service provided by Amazon to complete the task.

As we only skimmed the most basic configuration options in Amazon EC2, we recommend going back and investigating the different options available to you in Amazon ```EC2```. Of particular interest to you should be the different images available as some are specialized and come with machine learning libraries pre-installed. You should also familiarize yourself with [pricing](https://aws.amazon.com/ec2/pricing/on-demand/) for on-demand resources. And remember, turn off your instance if it's not doing anything! You can find instructions for terminating your EC2 instances [here](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/terminating-instances.html).